In [ ]:
pip install rtdl_revisiting_models


In [ ]:
pip install pytorch_tabnet

In [ ]:
# ===============================================================
# HIGGS BOSON — FTTransformer + SupCon + LGBM + XGB + TabNet + Stacking
# ===============================================================

import os, math, zipfile
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from rtdl_revisiting_models import FTTransformer
from pytorch_tabnet.tab_model import TabNetClassifier  # ✅ Added TabNet

# ---------------- Settings ----------------
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)

BATCH_SIZE = 512
EPOCHS = 40
LR = 7e-4
WEIGHT_DECAY = 3e-5
PROJ_DIM = 64

SUPCON_TEMPERATURE = 0.08
HARD_NEG_TOPK = 30
INITIAL_SUP_W, MAX_SUP_W = 0.12, 0.45

D_BLOCK = 128
N_BLOCKS = 4
ATTN_HEADS = 8
FFN_MULT = 4

N_FOLDS = 5

# ---------------- Kaggle File Handling ----------------
zip_files = {
    "train": "/kaggle/input/higgs-boson/training.zip",
    "test":  "/kaggle/input/higgs-boson/test.zip",
    "submission": "/kaggle/input/higgs-boson/random_submission.zip"
}
extract_dir = "/kaggle/working/higgs_data/"
os.makedirs(extract_dir, exist_ok=True)
for key, path in zip_files.items():
    if os.path.exists(path):
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(extract_dir)
            print(f"{key} unzipped.")
    else:
        print(f"{key} zip not found at {path}")

TRAIN_CSV = os.path.join(extract_dir, "training.csv")
TEST_CSV  = os.path.join(extract_dir, "test.csv")
OUT_SUB  = "/kaggle/working/submission.csv"

# ---------------- AMS Metric ----------------
def ams_score(s, b):
    b_reg = 10.0
    rad = 2.0 * ((s + b + b_reg) * math.log(1.0 + s / (b + b_reg)) - s)
    return math.sqrt(rad) if rad > 0 else 0.0

# ---------------- Dataset ----------------
class HiggsDataset(Dataset):
    def __init__(self, X, y=None, sample_weight=None):
        self.X = X.astype(np.float32)
        self.y = y
        self.sample_weight = sample_weight
    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        if self.y is None:
            return x, torch.tensor(-1, dtype=torch.long), torch.tensor(0.0)
        y = torch.tensor(int(self.y[idx]), dtype=torch.long)
        w = torch.tensor(float(self.sample_weight[idx]) if self.sample_weight is not None else 1.0)
        return x, y, w

# ---------------- SupCon Loss ----------------
class HardNegSupConLoss(nn.Module):
    def __init__(self, temperature=0.08, top_k=30):
        super().__init__()
        self.temperature = temperature
        self.top_k = top_k
    def forward(self, features, labels):
        device = features.device
        labels = labels.contiguous().view(-1,1)
        bs = features.shape[0]
        sim = torch.div(torch.matmul(features, features.T), self.temperature)
        same = torch.eq(labels, labels.T).float().to(device)
        diag = torch.eye(bs, device=device)
        pos_mask = same - diag
        neg_mask = 1.0 - same
        sim_max, _ = torch.max(sim, dim=1, keepdim=True)
        sim_stable = sim - sim_max.detach()
        exp_sim = torch.exp(sim_stable)
        pos_exp = exp_sim * pos_mask
        neg_sim_masked = sim.clone()
        neg_sim_masked[neg_mask == 0] = -1e9
        eps = 1e-12
        k = min(self.top_k, max(1, int(neg_mask.sum(dim=1).max().item())))
        if k == 0:
            denom = pos_exp.sum(dim=1) + (exp_sim*(1.0 - diag - pos_mask)).sum(dim=1)
            log_prob = torch.log((pos_exp.sum(dim=1)+eps)/(denom+eps))
            valid = pos_mask.sum(dim=1)>0
            return -log_prob[valid].mean()
        topk_vals, topk_idx = torch.topk(neg_sim_masked, k=k, dim=1)
        sel_mask = torch.zeros_like(neg_mask)
        arange = torch.arange(bs, device=device).unsqueeze(1).expand(-1,k)
        sel_mask[arange.reshape(-1), topk_idx.reshape(-1)] = 1.0
        sel_mask = sel_mask * neg_mask
        sum_pos = pos_exp.sum(dim=1)
        sum_neg = (exp_sim * sel_mask).sum(dim=1)
        log_prob = torch.log((sum_pos + eps)/(sum_pos + sum_neg + eps))
        valid = pos_mask.sum(dim=1)>0
        return -log_prob[valid].mean()

# ---------------- FTTransformer + SupCon ----------------
class FTTransformerSupCon(nn.Module):
    def __init__(self, n_features, proj_dim=64):
        super().__init__()
        self.ft = FTTransformer(
            n_cont_features=n_features,
            cat_cardinalities=[],
            d_block=D_BLOCK,
            n_blocks=N_BLOCKS,
            attention_n_heads=ATTN_HEADS,
            ffn_d_hidden_multiplier=FFN_MULT,
            attention_dropout=0.2,
            ffn_dropout=0.2,
            residual_dropout=0.1,
            d_out=2
        )
        self.proj_head = nn.Sequential(nn.Linear(2,128), nn.ReLU(), nn.Linear(128,proj_dim))
    def forward(self, x, return_emb=False):
        out = self.ft(x, x_cat=None)
        if isinstance(out, tuple): out = out[0]
        emb = F.normalize(self.proj_head(out), dim=1)
        return (out, emb) if return_emb else out

# ---------------- Weighted Focal Loss ----------------
class WeightedFocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, logits, targets, sample_weight=None):
        ce = F.cross_entropy(logits, targets, reduction='none')
        p_t = torch.exp(-ce)
        focal = ((1 - p_t) ** self.gamma) * ce
        if sample_weight is not None:
            focal = focal * sample_weight
            return focal.sum() / (sample_weight.sum() + 1e-12)
        return focal.mean()

# ---------------- Load Data ----------------
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

train_df.replace(-999.0, np.nan, inplace=True)
test_df.replace(-999.0, np.nan, inplace=True)

for c in train_df.columns:
    if c in ['EventId','Weight','Label']: continue
    if train_df[c].isna().any():
        train_df[c+'_miss'] = train_df[c].isna().astype(int)
        test_df[c+'_miss']  = test_df[c].isna().astype(int)

numeric_cols = [c for c in train_df.select_dtypes(include=np.number).columns if c != "Weight"]
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
num_cols_test = [c for c in numeric_cols if c in test_df.columns]
test_df[num_cols_test] = test_df[num_cols_test].fillna(train_df[num_cols_test].median())

# Add derived features
if {'DER_mass_MMC','DER_mass_vis'}.issubset(train_df.columns):
    train_df['mass_ratio'] = train_df['DER_mass_MMC']/(train_df['DER_mass_vis']+1e-6)
    test_df['mass_ratio']  = test_df['DER_mass_MMC']/(test_df['DER_mass_vis']+1e-6)
if {'PRI_tau_pt','PRI_met'}.issubset(train_df.columns):
    train_df['pt_ratio'] = train_df['PRI_tau_pt']/(train_df['PRI_met']+1e-6)
    test_df['pt_ratio']  = test_df['PRI_tau_pt']/(test_df['PRI_met']+1e-6)

y = (train_df['Label'] == 's').astype(int).values
weights = train_df['Weight'].values
event_ids_test = test_df['EventId'].values

train_features = train_df.drop(columns=['EventId','Weight','Label'], errors='ignore')
test_features  = test_df.drop(columns=['EventId'], errors='ignore')

scaler = StandardScaler()
X = scaler.fit_transform(train_features.values.astype(np.float32))
X_test = scaler.transform(test_features.values.astype(np.float32))

kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# ---------------- Containers ----------------
oof_nn, oof_lgb, oof_xgb, oof_tab = [np.zeros(len(X)) for _ in range(4)]
test_pred_nn = np.zeros(len(X_test))
test_pred_lgb_folds, test_pred_xgb_folds, test_pred_tab_folds = [], [], []

supcon = HardNegSupConLoss(temperature=SUPCON_TEMPERATURE, top_k=HARD_NEG_TOPK)

# ---------------- 1. FTTransformer + SupCon ----------------
for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
    print(f"\n===== NN Fold {fold+1}/{N_FOLDS} =====")
    X_tr, X_va = X[tr_idx], X[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    w_tr, w_va = weights[tr_idx], weights[va_idx]
    tr_loader = DataLoader(HiggsDataset(X_tr, y_tr, w_tr), batch_size=BATCH_SIZE, shuffle=True)
    va_loader = DataLoader(HiggsDataset(X_va, y_va, w_va), batch_size=BATCH_SIZE, shuffle=False)
    model = FTTransformerSupCon(X.shape[1], PROJ_DIM).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    criterion_cls = WeightedFocalLoss(alpha=0.75, gamma=2.0)
    best_auc, best_state = 0, None
    for epoch in range(EPOCHS):
        model.train()
        frac = epoch/max(1,EPOCHS-1)
        sup_w = INITIAL_SUP_W+(MAX_SUP_W-INITIAL_SUP_W)*frac
        for xb,yb,wb in tqdm(tr_loader, desc=f"Fold{fold+1} Epoch{epoch+1}", leave=False):
            xb,yb,wb=xb.to(DEVICE),yb.to(DEVICE),wb.to(DEVICE)
            optimizer.zero_grad()
            logits, emb = model(xb, return_emb=True)
            loss = criterion_cls(logits, yb, wb) + sup_w * supcon(emb, yb)
            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0); optimizer.step()
        scheduler.step()
        model.eval(); val_probs=[]
        with torch.no_grad():
            for xb,_,_ in va_loader:
                p=F.softmax(model(xb.to(DEVICE)),dim=1)[:,1].cpu().numpy(); val_probs.extend(p)
        val_auc=roc_auc_score(y_va,val_probs)
        if val_auc>best_auc: best_auc,val_probs_state=val_auc,val_probs; best_state={k:v.cpu() for k,v in model.state_dict().items()}
    model.load_state_dict(best_state)
    oof_nn[va_idx]=val_probs_state
    model.eval(); preds_test_fold=[]
    test_loader=DataLoader(HiggsDataset(X_test),batch_size=1024,shuffle=False)
    with torch.no_grad():
        for xb,_,_ in test_loader:
            p=F.softmax(model(xb.to(DEVICE)),dim=1)[:,1].cpu().numpy(); preds_test_fold.extend(p)
    test_pred_nn+=np.array(preds_test_fold)/N_FOLDS

# ---------------- 2. LightGBM ----------------
print("\n=== LightGBM ===")
for tr, va in kf.split(X, y):
    lgbm = lgb.LGBMClassifier(n_estimators=1200, learning_rate=0.01, num_leaves=64,
                              subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0)
    lgbm.fit(X[tr], y[tr], eval_set=[(X[va], y[va])],
             callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)])
    oof_lgb[va]=lgbm.predict_proba(X[va])[:,1]; test_pred_lgb_folds.append(lgbm.predict_proba(X_test)[:,1])
test_pred_lgb=np.mean(test_pred_lgb_folds,axis=0)

# ---------------- 3. XGBoost ----------------
print("\n=== XGBoost ===")
for tr, va in kf.split(X, y):
    xgbm = xgb.XGBClassifier(n_estimators=1200, learning_rate=0.01, max_depth=6,
                             subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                             eval_metric='auc', tree_method='hist', use_label_encoder=False)
    xgbm.fit(X[tr], y[tr], eval_set=[(X[va], y[va])], early_stopping_rounds=50, verbose=100)
    oof_xgb[va]=xgbm.predict_proba(X[va])[:,1]; test_pred_xgb_folds.append(xgbm.predict_proba(X_test)[:,1])
test_pred_xgb=np.mean(test_pred_xgb_folds,axis=0)


# ---------------- 4. TabNet ----------------
print("\n=== TabNet ===")
for tr, va in kf.split(X, y):
    tabnet = TabNetClassifier(
        n_d=16, n_a=16, n_steps=5, gamma=1.5,
        lambda_sparse=1e-4,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-3),
        mask_type='sparsemax',
        seed=SEED,
        verbose=10  # ✅ moved here instead of .fit()
    )

    tabnet.fit(
        X_train=X[tr], y_train=y[tr],
        eval_set=[(X[va], y[va])],
        patience=50, max_epochs=200, batch_size=1024, virtual_batch_size=128,
        eval_metric=['auc']
    )

    oof_tab[va] = tabnet.predict_proba(X[va])[:, 1]
    test_pred_tab_folds.append(tabnet.predict_proba(X_test)[:, 1])

test_pred_tab = np.mean(test_pred_tab_folds, axis=0)


# ---------------- Stacking ----------------
print("\n=== Meta-learner stacking ===")
stack_oof=np.vstack([oof_nn,oof_lgb,oof_xgb,oof_tab]).T
stack_test=np.vstack([test_pred_nn,test_pred_lgb,test_pred_xgb,test_pred_tab]).T
meta=LogisticRegression(max_iter=2000)
meta.fit(stack_oof, y, sample_weight=weights)
oof_meta=meta.predict_proba(stack_oof)[:,1]
final_test_pred=meta.predict_proba(stack_test)[:,1]
print("Stacked CV AUC:", roc_auc_score(y, oof_meta))

# ---------------- AMS ----------------
thr_range=np.linspace(0.01,0.99,99)
best_thr,best_ams=0.5,-1
for t in thr_range:
    s=weights[(y==1)&(oof_meta>t)].sum()
    b=weights[(y==0)&(oof_meta>t)].sum()
    sc=ams_score(s,b)
    if sc>best_ams: best_ams,best_thr=sc,t
print(f"Best AMS on stacked OOF = {best_ams:.3f} @ thr={best_thr:.4f}")

# ---------------- Submission ----------------
print("\nWriting submission...")
rankorder=np.argsort(np.argsort(final_test_pred))+1
classes=np.where(final_test_pred>best_thr,'s','b')
sub=pd.DataFrame({"EventId":event_ids_test,"RankOrder":rankorder,"Class":classes})
sub.to_csv(OUT_SUB,index=False)
print("Saved submission to:",OUT_SUB)
print("Final Stacked AMS:",best_ams)
